In [9]:
import requests

html = requests.get("https://keithgalli.github.io/web-scraping/example.html")

print(html.content)

b'<html>\n<head>\n<title>HTML Example</title>\n</head>\n<body>\n\n<div align="middle">\n<h1>HTML Webpage</h1>\n<p>Link to more interesting example: <a href="https://keithgalli.github.io/web-scraping/webpage.html">keithgalli.github.io/web-scraping/webpage.html</a></p>\n</div>\n\n<h2>A Header</h2>\n<p><i>Some italicized text</i></p>\n\n<h2>Another header</h2>\n<p id="paragraph-id"><b>Some bold text</b></p>\n\n</body>\n</html>\n'


In [10]:
print("Response url:",html.url)

print("Response status:",html.status_code)

print("Response status:",html.encoding)

Response url: https://keithgalli.github.io/web-scraping/example.html
Response status: 200
Response status: utf-8


In [11]:
print("Response headers:",html.headers)

Response headers: {'Connection': 'keep-alive', 'Content-Length': '259', 'Server': 'GitHub.com', 'Content-Type': 'text/html; charset=utf-8', 'permissions-policy': 'interest-cohort=()', 'Last-Modified': 'Fri, 03 Jul 2020 00:21:07 GMT', 'Access-Control-Allow-Origin': '*', 'ETag': 'W/"5efe79f3-198"', 'expires': 'Sun, 05 Mar 2023 08:25:59 GMT', 'Cache-Control': 'max-age=600', 'Content-Encoding': 'gzip', 'x-proxy-cache': 'MISS', 'X-GitHub-Request-Id': '478A:5E4B:69586C:8E6CB7:64044FBF', 'Accept-Ranges': 'bytes', 'Date': 'Sun, 05 Mar 2023 08:30:33 GMT', 'Via': '1.1 varnish', 'Age': '0', 'X-Served-By': 'cache-bkk2310026-BKK', 'X-Cache': 'HIT', 'X-Cache-Hits': '1', 'X-Timer': 'S1678005033.222481,VS0,VE275', 'Vary': 'Accept-Encoding', 'X-Fastly-Request-ID': 'b0e3908d460e6ad022d7d056fffa6a97a8377333'}


# ดึงข้อมูลตรวจสลากกินแบ่งรัฐบาล (แบบง่าย)

In [25]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

html = requests.get("https://news.sanook.com/lotto/check/16102563/")
bs = BeautifulSoup(html.content,"html.parser")

In [26]:
def parse_lotto_results(bs):
    result_dict = {"number1": "", "number1Nearby": [], "lastThreeDigit": [], "lastTwodigit": "", "secondPrice": [], "thirdPrice": [], "fourthPrice": [], "fifthPrice": []}
    
    lotto_sections = bs.find_all("div", attrs={'class': 'lottocheck__sec lottocheck__sec--bdnone'})
    for section in lotto_sections:
        table = ["lottocheck__table","lottocheck__sec--nearby"]
        for tab in table:
            lotto_tables = section.find_all("div", attrs={"class": f"{tab}"})
            lotto_numbers = [number.text for table in lotto_tables for number in table.find_all("strong", attrs={"class": "lotto__number"})]

            for num in lotto_numbers:
                if len(num) == 6:
                    if tab == "lottocheck__sec--nearby":
                        result_dict["number1Nearby"].append(num)
                    else:
                        result_dict["number1"] = num

                elif len(num) == 3:
                    result_dict["lastThreeDigit"].append(num)

                elif len(num) == 2:
                    result_dict["lastTwodigit"] = num

    lotto = bs.find_all("div", attrs={'class': 'lottocheck__box-item'})
    name = ["secondPrice","thirdPrice","fourthPrice","fifthPrice"]
    count = [5,10,50,100]
    c = 1
    l = 0
    for i in range(len(lotto)):
        n = lotto[i].find_all("span", attrs={'class': 'lotto__number'})
        for j in range(len(n)):
            if c <= count[l]:
                result_dict[name[l]].append(n[j].text)
                c += 1
            else:
                l += 1
                c = 1
                result_dict[name[l]].append(n[j].text)
                c += 1

    return result_dict


In [37]:
lotto = pd.DataFrame({"Link":[],"date":[]})


In [45]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Define an empty list to hold the scraped data
data = []

# Make the request and parse the HTML using BeautifulSoup
url = "https://news.sanook.com/lotto/archive/"
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

# Find all the option elements within the lotto-form divs
lotto_forms = soup.find_all("div", attrs={"class": "lotto-form"})
for form in lotto_forms:
    options = form.find_all("option")
    for option in options:
        # Extract the link and date values from each option element
        link = option["value"]
        date = option.text.strip()

        # Append the data as a dictionary to the list
        data.append({"Link": link, "date": date})

# Create the dataframe from the scraped data
lotto = pd.DataFrame(data)


In [46]:
lotto

,Link,date
0,01032566,1 มีนาคม 2566
1,16022566,16 กุมภาพันธ์ 2566
2,01022566,1 กุมภาพันธ์ 2566
3,17012566,17 มกราคม 2566
4,30122565,30 ธันวาคม 2565
5,16122565,16 ธันวาคม 2565
6,01122565,1 ธันวาคม 2565
7,16112565,16 พฤศจิกายน 2565
8,01112565,1 พฤศจิกายน 2565
9,16102565,16 ตุลาคม 2565


In [47]:
result_dict = parse_lotto_results(bs)
result_dict

{'number1': '',
 'number1Nearby': [],
 'lastThreeDigit': [],
 'lastTwodigit': '',
 'secondPrice': [],
 'thirdPrice': [],
 'fourthPrice': [],
 'fifthPrice': []}